In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [44]:
df = pd.read_csv("Corona.csv")

In [45]:
df.head()

,Ind_ID,Test_date,Cough_symptoms,Fever,Sore_throat,Shortness_of_breath,Headache,Corona,Age_60_above,Sex,Known_contact
0,1,11-03-2020,TRUE,FALSE,TRUE,FALSE,FALSE,negative,None,None,Abroad
1,2,11-03-2020,FALSE,TRUE,FALSE,FALSE,FALSE,positive,None,None,Abroad
2,3,11-03-2020,FALSE,TRUE,FALSE,FALSE,FALSE,positive,None,None,Abroad
3,4,11-03-2020,TRUE,FALSE,FALSE,FALSE,FALSE,negative,None,None,Abroad
4,5,11-03-2020,TRUE,FALSE,FALSE,FALSE,FALSE,negative,None,None,Contact with confirmed


In [46]:
df.shape

(278848, 11)

In [47]:
df.nunique()

Ind_ID                 278848
Test_date                  51
Cough_symptoms              5
Fever                       5
Sore_throat                 5
Shortness_of_breath         5
Headache                    5
Corona                      3
Age_60_above                3
Sex                         3
Known_contact               3
dtype: int64

In [48]:
df.drop(["Ind_ID","Test_date"], axis=1, inplace = True)

In [49]:
df.replace("None", np.nan, inplace=True)
df = df.replace("other", np.nan)

In [50]:
df.isnull().sum()

Cough_symptoms            252
Fever                     252
Sore_throat                 1
Shortness_of_breath         1
Headache                    1
Corona                   3892
Age_60_above           127320
Sex                     19563
Known_contact               0
dtype: int64

In [51]:
for i in df.columns:
  print("\nUnique values in '{}' are :\n".format(i),df[i].unique())


Unique values in 'Cough_symptoms' are :
 ['TRUE' 'FALSE' nan False True]

Unique values in 'Fever' are :
 ['FALSE' 'TRUE' nan False True]

Unique values in 'Sore_throat' are :
 ['TRUE' 'FALSE' nan False True]

Unique values in 'Shortness_of_breath' are :
 ['FALSE' 'TRUE' nan False True]

Unique values in 'Headache' are :
 ['FALSE' 'TRUE' nan False True]

Unique values in 'Corona' are :
 ['negative' 'positive' nan]

Unique values in 'Age_60_above' are :
 [nan 'No' 'Yes']

Unique values in 'Sex' are :
 [nan 'male' 'female']

Unique values in 'Known_contact' are :
 ['Abroad' 'Contact with confirmed' 'Other']


In [52]:
df.shape

(278848, 9)

In [53]:
df["Cough_symptoms"] = df["Cough_symptoms"].replace("TRUE",1)
df["Cough_symptoms"] = df["Cough_symptoms"].replace("FALSE",0)

In [54]:
df["Fever"] = df["Fever"].replace("TRUE",1)
df["Fever"] = df["Fever"].replace("FALSE",0)

In [55]:
df["Sore_throat"] = df["Sore_throat"].replace("TRUE",1)
df["Sore_throat"] = df["Sore_throat"].replace("FALSE",0)

In [56]:
df["Shortness_of_breath"] = df["Shortness_of_breath"].replace("TRUE",1)
df["Shortness_of_breath"] = df["Shortness_of_breath"].replace("FALSE",0)

In [57]:
df["Headache"] = df["Headache"].replace("TRUE",1)
df["Headache"] = df["Headache"].replace("FALSE",0)

In [58]:
df["Corona"] = df["Corona"].replace("positive",1)
df["Corona"] = df["Corona"].replace("negative",0)

In [59]:
df["Age_60_above"] = df["Age_60_above"].replace("Yes",1)
df["Age_60_above"] = df["Age_60_above"].replace("No",0)

In [60]:
df["Sex"] = df["Sex"].replace("male",1)
df["Sex"] = df["Sex"].replace("female",0)

In [61]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
work_reshaped = np.array(df['Known_contact']).reshape(-1, 1)
xx = np.array(ct.fit_transform(work_reshaped))
print(xx)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [62]:
work_reshape = pd.DataFrame(xx, columns=['Abroad', 'Contact with confirmed', 'Other'])
df1 = pd.concat([work_reshape, df], axis = 1)

In [63]:
df1.drop(["Known_contact"], axis=1, inplace = True)
df1.head()

,Abroad,Contact with confirmed,Other,Cough_symptoms,Fever,Sore_throat,Shortness_of_breath,Headache,Corona,Age_60_above,Sex
0,1.0,0.0,0.0,1,0,1,0,0,0.0,NaN,NaN
1,1.0,0.0,0.0,0,1,0,0,0,1.0,NaN,NaN
2,1.0,0.0,0.0,0,1,0,0,0,1.0,NaN,NaN
3,1.0,0.0,0.0,1,0,0,0,0,0.0,NaN,NaN
4,0.0,1.0,0.0,1,0,0,0,0,0.0,NaN,NaN


In [64]:
df1.dropna(inplace = True)

In [65]:
df1.shape

(136294, 11)

In [66]:
X = df1[["Abroad","Contact with confirmed", "Other" , "Cough_symptoms", "Fever", "Sore_throat", "Shortness_of_breath", "Headache"]]
y = df1["Corona"]

In [67]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

X = preprocessor.fit_transform(X)

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [69]:
X_train.shape, y_train.shape

((102220, 13), (102220,))

In [70]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)
y_pred = log_reg.predict(X_test)
print("Accuracy of Logistic Regression Model : {}%".format(round(accuracy_score(y_pred, y_test)*100)))
print("Classification Report of Logistic Regression Model : \n",classification_report(y_pred, y_test))

Accuracy of Logistic Regression Model : 95%
Classification Report of Logistic Regression Model : 
               precision    recall  f1-score   support

         0.0       0.99      0.96      0.97     32335
         1.0       0.50      0.74      0.59      1739

    accuracy                           0.95     34074
   macro avg       0.74      0.85      0.78     34074
weighted avg       0.96      0.95      0.95     34074



In [71]:
from sklearn.model_selection import GridSearchCV
parameter = {"penalty":["l1", "l2", "elasticnet"],
             "C" : [1,2,3,4,5,6,10,20,40,50],
             "max_iter" : [100,200,300,400,500]}
Classfier_Reg = GridSearchCV(log_reg,param_grid=parameter, scoring="accuracy", cv=10)

In [72]:
Classfier_Reg.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': [1, 2, 3, 4, 5, 6, 10, 20, 40, 50],
                         'max_iter': [100, 200, 300, 400, 500],
                         'penalty': ['l1', 'l2', 'elasticnet']},
             scoring='accuracy')

In [73]:
print(Classfier_Reg.best_params_)
print(Classfier_Reg.best_score_)
y_pred_tune_grid = Classfier_Reg.predict(X_test)
print("Accuracy is {}%".format(round(accuracy_score(y_pred_tune_grid, y_test)*100)))
print(classification_report(y_pred_tune_grid, y_test))

{'C': 1, 'max_iter': 100, 'penalty': 'l2'}
0.9467618861279593
Accuracy is 95%
              precision    recall  f1-score   support

         0.0       0.99      0.96      0.97     32335
         1.0       0.50      0.74      0.59      1739

    accuracy                           0.95     34074
   macro avg       0.74      0.85      0.78     34074
weighted avg       0.96      0.95      0.95     34074



In [74]:
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(X_train,y_train)
y_pred_dec = dec_tree.predict(X_test)
accuracy_score(y_pred_dec, y_test)*100

95.9529259846217

In [75]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
y_pred_knn = knn.predict(X_test)
accuracy_score(y_pred_knn, y_test)*100

95.94412161765568

In [76]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier()
random_forest.fit(X_train,y_train)
y_pred_rfc = random_forest.predict(X_test)
accuracy_score(y_pred_rfc, y_test)*100

95.9529259846217

In [77]:
from sklearn.ensemble import GradientBoostingClassifier
grad_boost = GradientBoostingClassifier()
grad_boost.fit(X_train,y_train)
y_pred_gbc = grad_boost.predict(X_test)
accuracy_score(y_pred_gbc, y_test)*100

95.95586077361038

In [78]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train,y_train)
y_pred_svm = svm.predict(X_test)
accuracy_score(y_pred_svm, y_test)*100

95.95586077361038